<a href="https://colab.research.google.com/github/avkornaev/Cells_counter/blob/main/01CC_Normalization_parameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Summary.

The cells counter very first step is calculation of the normalization parameters.

#**Install libraries**.



In [1]:
!pip install -Uqq fastbook
!pip install czifile
!pip install tifffile
!pip install imagecodecs

     |████████████████████████████████| 720 kB 6.6 MB/s 
     |████████████████████████████████| 46 kB 975 kB/s 
     |████████████████████████████████| 1.2 MB 44.7 MB/s 
     |████████████████████████████████| 189 kB 67.6 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
     |████████████████████████████████| 51 kB 346 kB/s 
     |████████████████████████████████| 29.6 MB 60 kB/s 


#**Connect to Google disk.**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Import libraries.**

In [3]:
import fastbook
from fastbook import *
import cv2
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
from google.colab.patches import cv2_imshow
from scipy import ndimage
from skimage import measure, color, io, img_as_ubyte, img_as_float
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.segmentation import clear_border, join_segmentations
import pandas as pd
import czifile
import imagecodecs
#import tifffile

#**Check the settings.**

In [4]:
print(os.getcwd()) # current directory

/content


In [5]:
#Image under study
#path = Path('/content/drive/MyDrive/Cells_counter/Segmentation_Train_and_Validation_dataset/Images')
path = Path('K:\Cells_counter\Init_Images')
#Visualisation
img_size = [] # empty [] or rectangle coordinates, e.g. [0,0,1024,1024]
fig_size = (30,30) # figures size
img_map  = 'Blues' # image map. For example, 'gray','hot','jet','Blues' et al. If you want to know more, please type the following code: help(plt.imshow)
img_map1 = 'jet'

font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
fontColor = (255,255,255)
lineType  = 2

#**Download "czi" images and create slices. Then calculate mean and standard deviation values.**

Path content.

In [7]:
path_list = path.ls()

Read an example of an image in the dataset.

In [39]:
img = czifile.imread(path_list[-1])
img.shape

(1, 1, 1, 2, 1, 3277, 3277, 1)

In [40]:
img = czifile.imread(path_list[1])
img_size = [0,0,img.shape[5],img.shape[6]] # full size
img_slice = img[0,0,0,:,0,img_size[0]:img_size[2],img_size[1]:img_size[3],0]# image in a form of 3D matrix

img0 = img_slice[0,:,:] # 0 channel which is fluo4-T1 (highlights the apoptosed cells)
img1 = img_slice[1,:,:] # 1 channel which is H3342-T2 (highlights all the cells)
img2 = np.uint16(np.zeros(img0.shape))# 2 empty channel
channels = [img0,img1,img2] # channels of the image

nc = len(channels)
n = [i for i in range(nc)] # auxiliary variable

print('channel shape: ',channels[2].shape, channels[2].dtype, type(channels[2]))

channel shape:  (3277, 3277) uint16 <class 'numpy.ndarray'>


In [41]:
print(np.mean(img0), np.max(img0), np.min(img0), np.std(img0))
print(np.mean(img1), np.max(img1), np.min(img1), np.std(img1))

993.4179357724736 65535 0 5065.741187444674
932.4809735863528 65535 87 5626.293935061123


Read all images in the path and calculate mean and std values.

In [12]:
#img_tensors = [tensor(czifile.imread(path_list[o])) for o in path_list]

In [17]:
mean_values = [0]*nc
std_values = [0]*nc
imgf = [0]*nc

for i in range(len(path.ls())): #loop through the images path
  img = czifile.imread(path_list[i]) # read czi image file
  img_size = [0,0,img.shape[5],img.shape[6]] # full size
  img_slice = img[0,0,0,:,0,img_size[0]:img_size[2],img_size[1]:img_size[3],0]# image in a form of 3D matrix
  imgf[0] = img_slice[0,:,:] # 0 channel which is fluo4-T1 (highlights the apoptosed cells)
  imgf[1] = img_slice[1,:,:] # 1 channel which is H3342-T2 (highlights all the cells)
  imgf[2] = np.uint16(np.zeros(imgf[0].shape))# 2 empty channel
  
  for j in n:
    if i < 1:
      channels[j] = np.reshape(imgf[j],(1,-1)) # channels of the image
    else:
      #print(channels[j].shape, imgf[j].shape)
      channels[j] = np.concatenate([channels[j],np.reshape(imgf[j],(1,-1))])
  print(channels[j].shape)    

(1, 10738729)
(2, 10738729)
(3, 10738729)
(4, 10738729)
(5, 10738729)


In [ ]:
aaa =  np.reshape(channels[0],(-1,1))
print(aaa.shape,aaa.dtype)
std_valuesa  =  np.std(np.reshape(channels[0],(1,-1)))


In [29]:
for j in n:
  mean_values[j] = np.mean(np.reshape(channels[j],(1,-1)))#/np #mean value
  std_values[j]  =  np.std(np.reshape(channels[j],(1,-1)))#/np standard deviation

print (mean_values, std_values)

[375.3508845413643, 774.9535605005025, 0.0] [2617.964987351893, 4097.3656955349425, 0.0]
